<h1><center> Aggregate CPS Data </center></h1>


data dictionary [here](https://www2.census.gov/programs-surveys/cps/datasets/2022/basic/2022_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt)

data instruction [here](https://www2.census.gov/programs-surveys/cps/methodology/PublicUseDocumentation_final.pdf)

In this file, we will aggregate the data to state level. The equations for estimates at the population are provided in data instruction page 9. A more detailed methodology for weights and estimation can be found [here Page 67](https://www2.census.gov/programs-surveys/cps/methodology/CPS-Tech-Paper-77.pdf) Tasks in the following code can be described as:
1. Import the cleaned, concatenated csv file
    - We might wondering about the differences in calculation between the raw data and the cleaned data. Theoratically, there should be no difference. The same code is applied to the raw dataset at the end for comparison.
2. Estimation calculation by state
    - Time span: Yearly, Monthly
    - Estimations: Labor force related; The estimates include:
        - Civil Noninstitutional Population (number)
        - Labor Force (number)
        - Employment (number)
        - Unemployment (number)
        - Labor Force Participation Rate (percentage)
        - Employment Rate (percentage)
        - Unemployment Rate (percentage)
        - Employment-Population Ratio (percentage)
    - Those info are stored in a new dataframe and export as csv file
    - Verify the calculation
        - If we compare the calculation with BLS results [employment status of the civilian noninstitutional population(not seasonally adjusted)](https://www.bls.gov/web/laus/ststdnsadata.txt) produced by [LAUS](https://www.bls.gov/lau/rdscnp16.htm), we will find some discrepancy which does not come from the use of the CPS microdata.It comes more fundamentally from the fact that, unlike with the national-level unemployment rate, BLS’s state-level labor force statistics are not directly calculated from CPS data. Instead, we use a signal-and-noise model with covariates in order to estimate the state-level employment and unemployment levels. This model helps us limit the potential error from monthly sampling variability within the state-level CPS data, and thus better determine whether monthly changes in the levels are from actual changes in employment/unemployment or from sampling variability. From these model-based levels, the unemployment rate is then computed. Therefore, the official model-based unemployment rates for the states will be close but not generally identical to the ones you can calculate directly from the microdata. More information about our methods is available [here](https://www.bls.gov/lau/laumthd.htm) or from our chapter in the BLS Handbook of Methods [here](https://www.bls.gov/opub/hom/lau/home.htm).
3. Other estimations
    - Besides the labor force estimates in step2, other information can be aggregated as well. Still, group by month and state
    - Those estimates include:
        - Demographics (percentage), such as AgeGroup, Gender, Citizenship, etc
        - Union (percentage)

In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# current working directory should be `./code`, check it:
print(os.getcwd())

GLOBAL_PATH = '../'
RAW_DATA_PATH = 'raw_data/'
PROCESSED_DATA_PATH = 'processed_data/'

# specify the year of CPS data
CPS_YEAR = 'cps2021'

/Users/niksun/Desktop/data-repository-partial-code/code


In [2]:
df = pd.read_csv(GLOBAL_PATH + PROCESSED_DATA_PATH + str(CPS_YEAR) + '.csv')
df.head()

,hrhhid,hrhhid2,PULINENO,Year,Month,MonthInSample,State,County,Metropolitan,Age,Marital,Gender,Educ,Race,Hispanic,Citizen,FamIncome,LaborForce,FullPartTime,WeeklyEarning,flagWeekEarn,HourlyEarning,flagPeriodicityEarn,flagConfirmHourly,SingleJob,NumJobs,UsualHours,ActualHours,Union,Occupation,WorkerClass,hwhhwgt,pwlgwgt,pworwgt,pwvetwgt,pwsswgt,pwcmpwgt,AgeGroup
0,505019880110916,11011,2.0,2021,7,8,1,3,1.0,61.0,1.0,0.0,4.0,0.0,0.0,1.0,12,5.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,19202002.0,28699695.0,76121915.0,18945420.0,19202002.0,18976311.0,old
1,610100690316751,11011,1.0,2021,7,6,1,3,1.0,53.0,1.0,0.0,4.0,0.0,0.0,1.0,9,1.0,2.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,50.0,60.0,-1.0,1.0,5.0,20812987.0,31107507.0,0.0,21153559.0,20812987.0,21204165.0,prime
2,180314039113,13011,1.0,2021,7,2,1,0,1.0,77.0,1.0,0.0,1.0,0.0,0.0,1.0,8,5.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,18287603.0,27333017.0,0.0,18351717.0,18287603.0,18396932.0,retired
3,310588190701104,11011,1.0,2021,7,8,1,0,0.0,85.0,1.0,0.0,0.0,0.0,0.0,1.0,6,5.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,18424472.0,27537584.0,74350105.0,18489066.0,18424472.0,18534619.0,retired
4,2790110012113,11011,1.0,2021,7,7,1,0,0.0,73.0,1.0,1.0,1.0,1.0,0.0,1.0,10,1.0,2.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,40.0,40.0,-1.0,2.0,4.0,17561071.0,26267955.0,0.0,17099389.0,17561071.0,17124931.0,retired


## Step 1: Labor Force Estimations

### 1.1 Civil Noninstitutional Population (Number)

When we consider labor force and employment, we only count the population that are at least 16 years old and not serving the arm force. This group of people are called Civil Noninstitutional Population.

$ Civil\_Noninstitutional\_Population\_yearly = \frac{1}{12} * \sum_i wgt_i * P_i $
- If we want to calculate the estimation for the year, we need to 
    - divide the yearly population by 12
    - use the concat dataframe from Jan to Dec

The monthly number of civil noninstitution population is:

$$ Civil\_Noninstitutional\_Population = \sum_i wgt_i * P_i $$
- $P_i$: 1 if individual $i$ is civil noninstitutional population and 0 otherwise
    - this indicator should be 1 for all observations in the cleaned dataframe (I kept only those people in dataset)
- $wgt_i$: composite final weight `PWCMPWGT` of individual $i$, divided by 10,000

In [15]:
# calculate the civil noninstitutioanl population for each month, each state

# create civil noninstitutional population indicator P, initialize to 1 and should be 1 for all obs
df['P'] = 1
# create a new column to calculate P_i * wgt_i / 10000
df['Civil_Pop'] = df['P'] * df['pwcmpwgt'] / 10_000
print("Total Number of Civil Noninstitutional Population is: ", df['Civil_Pop'].sum())
# group by month and state, sum up the Civil_Pop, store in a df_civil dataframe
df_civil = df.groupby(['Month', 'State']).sum(numeric_only=True)['Civil_Pop'].reset_index()
print("Shape of df_civil is: ", df_civil.shape)
if df_civil.shape[0] == 12 * 51:
    print("All months and states are included in df_civil")
else:
    print("Missing months or states in df_civil")
df_civil.head(3)

Total Number of Civil Noninstitutional Population is:  3137341859.0018
Shape of df_civil is:  (612, 3)
All months and states are included in df_civil


,Month,State,Civil_Pop
0,1,1,3.890500e+06
1,1,2,5.448940e+05
2,1,4,5.932506e+06


### 1.2 Labor Force (Number)

Within the Civil Noninstitutional Population, we can further divide them into labor force and non-labor force. The labor force is defined as the people who are at least 16 years old and not serving the arm force, and are either employed or unemployed.

The monthly number of people in labor force is:
$$ Labor\_Force = \sum_i wgt_i * L_i \$$
- $L_i$: 1 if individual $i$ is in labor force, 0 otherwise. Identify in labor force: $1 \leq LaborForce \leq 4$
- $wgt_i$: composited final weight `PWCMPWGT` of person i, divided by 10,000

In [16]:
# calculate the labor force population for each month, each state

# create indicator L, initialize to 0, recode to 1 if LaborForce between 1 and 4
df['L'] = np.where((df['LaborForce'] >= 1) & (df['LaborForce'] <= 4), 1, 0)
# calculate the number of labored people
df['LaborForce_Pop'] = df['L'] * df['pwcmpwgt'] / 10_000
print("Total Number of Labor Force Population is: ", df['LaborForce_Pop'].sum())
# group by month and state, sum up the LaborForce_Pop, store in a df_labor dataframe
df_labor = df.groupby(['Month', 'State']).sum(numeric_only=True)['LaborForce_Pop'].reset_index()
print("Shape of df_labor is: ", df_labor.shape)
if df_labor.shape[0] == 12 * 51:
    print("All months and states are included in df_labor")
else:
    print("Missing months or states in df_labor")
df_labor.head(3)

Total Number of Labor Force Population is:  1934446796.4306996
Shape of df_labor is:  (612, 3)
All months and states are included in df_labor


,Month,State,LaborForce_Pop
0,1,1,2.216382e+06
1,1,2,3.478068e+05
2,1,4,3.554388e+06


### 1.3 Unemployment (Number)

Based on the valid values of `LaborForce`, unemployment can be either on-layoff or actively looking for work.

The monthly number of people unemployed is:
$$ Unemployment = \sum_i wgt_i * U_i \$$
- $U_i$: 1 if individual $i$ is unemployed, 0 otherwise. Identify unemployed: $3 \leq LaborForce \leq 4$
- $wgt_i$: composited final weight `PWCMPWGT` of person i, divided by 10,000

In [17]:
# calculate the unemployed population for each month, each state

# create indicator L, initialize to 0, recode to 1 if LaborForce between 3 and 4
df['U'] = np.where((df['LaborForce'] >= 3) & (df['LaborForce'] <= 4), 1, 0)
# calculate the number of people who are unemployed
df['Unemp_Pop'] = df['U'] * df['pwcmpwgt'] / 10_000
print("Total Number of Unemployed Population is: ", df['Unemp_Pop'].sum())
# group by month and state, sum up the Unemp_Pop, store in a df_unemp dataframe
df_unemp = df.groupby(['Month', 'State']).sum(numeric_only=True)['Unemp_Pop'].reset_index()
print("Shape of df_unemp is: ", df_unemp.shape)
if df_unemp.shape[0] == 12 * 51:
    print("All months and states are included in df_unemp")
else:
    print("Missing months or states in df_unemp")
df_unemp.head(3)

Total Number of Unemployed Population is:  103478733.62200001
Shape of df_unemp is:  (612, 3)
All months and states are included in df_unemp


,Month,State,Unemp_Pop
0,1,1,98597.3449
1,1,2,26770.8435
2,1,4,237564.4493


### 1.4 Employment (Number)

Based on the valid values of `LaborForce`, employment can be either at_work or absent.

The monthly number of people employed is:
$$ Employment = \sum_i wgt_i * E_i \$$
- $E_i$: 1 if individual $i$ is employed, 0 otherwise. Identify employed: $1 \leq LaborForce \leq 2$
- $wgt_i$: composited final weight `PWCMPWGT` of person i, divided by 10,000

In [18]:
# calculate the employed population for each month, each state

# create indicator E, initialize to 0, recode to 1 if LaborForce between 1 and 2
df['E'] = np.where((df['LaborForce'] >= 1) & (df['LaborForce'] <= 2), 1, 0)
# calculate the number of people who are employed
df['Emp_Pop'] = df['E'] * df['pwcmpwgt'] / 10_000
print("Total Number of Employed Population is: ", df['Emp_Pop'].sum())
# group by month and state, sum up the Emp_Pop, store in a df_emp dataframe
df_emp = df.groupby(['Month', 'State']).sum(numeric_only=True)['Emp_Pop'].reset_index()
print("Shape of df_emp is: ", df_emp.shape)
if df_emp.shape[0] == 12 * 51:
    print("All months and states are included in df_emp")
else:
    print("Missing months or states in df_emp")

Total Number of Employed Population is:  1830968062.8087003
Shape of df_emp is:  (612, 3)
All months and states are included in df_emp


### 1.5 Merge the "Number" Estimates

From 1.1 to 1.4, we calculated four numbers: civil noninstitutional population, labor force, unemployment, and employment. We can merge them together to form a new dataframe.

In [19]:
# merge df_civil, df_labor, df_unemp, df_emp into one dataframe

# can i do this in one step???

# merge df_civil and df_labor
df_civil_labor = pd.merge(df_civil, df_labor, on=['Month', 'State'], how='left')
# merge df_unemp and df_emp
df_unemp_emp = pd.merge(df_unemp, df_emp, on=['Month', 'State'], how='left')
# merge all
df_labor_all = pd.merge(df_civil_labor, df_unemp_emp, on=['Month', 'State'], how='left')

df_labor_all.head(3)

,Month,State,Civil_Pop,LaborForce_Pop,Unemp_Pop,Emp_Pop
0,1,1,3.890500e+06,2.216382e+06,98597.3449,2.117785e+06
1,1,2,5.448940e+05,3.478068e+05,26770.8435,3.210359e+05
2,1,4,5.932506e+06,3.554388e+06,237564.4493,3.316824e+06


### 1.6 Rates

$$ Unemployment\_Rate = \frac{Unemployment}{Labor\_Force} =\frac{\sum_i wgt_i * U_i}{\sum_i wgt_i * L_i} * 100 $$

$$ Employment\_Rate = \frac{Employment}{Labor\_Force} =\frac{\sum_i wgt_i * E_i}{\sum_i wgt_i * L_i} * 100 $$

$$ Labor\_Force\_Participation\_Rate = \frac{Labor\_Force}{Civil\_Noninstitutional\_Population} = \frac{\sum_i wgt_i * L_i}{\sum_i wgt_i * P_i} * 100 $$

$$Employment\_Population\_Ratio = \frac{Employment}{Civil\_Noninstitutional\_Population} = \frac{\sum_i wgt_i * E_i}{\sum_i wgt_i * P_i} * 100 $$



In [20]:
df_labor_all['Emp_Rate'] = df_labor_all['Emp_Pop'] / df_labor_all['LaborForce_Pop'] * 100
df_labor_all['Unemp_Rate'] = df_labor_all['Unemp_Pop'] / df_labor_all['LaborForce_Pop'] * 100
df_labor_all['LF_Particip_Rate'] = df_labor_all['LaborForce_Pop'] / df_labor_all['Civil_Pop'] * 100
df_labor_all['Emp_Pop_Ratio'] = df_labor_all['Emp_Pop'] / df_labor_all['Civil_Pop'] * 100
df_labor_all.head(6)

,Month,State,Civil_Pop,LaborForce_Pop,Unemp_Pop,Emp_Pop,Emp_Rate,Unemp_Rate,LF_Particip_Rate,Emp_Pop_Ratio
0,1,1,3.890500e+06,2.216382e+06,9.859734e+04,2.117785e+06,95.551428,4.448572,56.969080,54.434770
1,1,2,5.448940e+05,3.478068e+05,2.677084e+04,3.210359e+05,92.302955,7.697045,63.830171,58.917134
2,1,4,5.932506e+06,3.554388e+06,2.375644e+05,3.316824e+06,93.316305,6.683695,59.913773,55.909319
3,1,5,2.365706e+06,1.333109e+06,5.702834e+04,1.276081e+06,95.722156,4.277844,56.351442,53.940815
4,1,6,3.109385e+07,1.861967e+07,1.715757e+06,1.690391e+07,90.785241,9.214759,59.882154,54.364158
5,1,8,4.642794e+06,3.167476e+06,1.957061e+05,2.971770e+06,93.821386,6.178614,68.223485,64.008220


## Step 2: Other Estimations

### 2.1 Union

Feedback from BLS:
1. You can view annual average data on union membership by state at [BLS Union Tables](https://www.bls.gov/webapps/legacy/cpslutab5.htm). The source of the state union membership data is the CPS. Also, as you probably know, an annual average is the 12-month average of not seasonally adjusted data. LAUS does not publish monthly state union membership data due to sample size limitations of the CPS. 

2. To tabulate data on union membership, you should use the labor force status recode (PEMLR = 1 and 2), class of worker variable (PEI01COW = 1-5), and union membership variable (PEERNLAB). In addition, you should use the outgoing rotation weight (PWORWGT) because information on union membership is only collected from a quarter of the CPS sample each month.

In data dictionary, the edited universe for Union (PEERNLAB) is: (PEIO1COW = 1-5 AND PEMLR = 1-2 AND HRMIS = 4, 8). And in the dataset, if MonthInSample (HRMIS) = 4, 8, LaborForce (PEMLR) = 1-2, WorkerClass (PEIO1COW) = 1-5, then Union (PEERNLAB) is valid (0 No 1 Yes after cleaning). Otherwise, Union (PEERNLAB) is -1 (meaning blank). The BLS response did not mention that I should take are of one of the condition: MonthInSample (HRMIS) = 4, 8. But it should be used.

$$ Union\_Member = \sum_i wgt_i * UM_i \$$
- $UM_i$: 1 if individual $i$ is union member, 0 otherwise. Identify union member: $Union == 1$
- $wgt_i$: outgoing rotation weight `PWORWGT` of person i, divided by 10,000

$$ Union\_Rate = \frac{Union\_Member}{Emp\_Pop} * 100 $$

In [21]:
# calculate members of union for each month, each state
# create indicator UM, initialize to 0, recode to 1 if Union is 1
df['UM'] = np.where(df['Union'] == 1, 1, 0)
# calculate the number of people who are members of union
df['Union_Pop'] = df['UM'] * df['pworwgt'] / 10_000
print("Total Number of Union Members is: ", df['Union_Pop'].sum())
# group by month and state, sum up the Union_Pop, store in a df_union dataframe
df_union = df.groupby(['Month', 'State']).sum(numeric_only=True)['Union_Pop'].reset_index()
print("Shape of df_union is: ", df_union.shape)
if df_union.shape[0] == 12 * 51:
    print("All months and states are included in df_union")
else:
    print("Missing months or states in df_union")
print(df_union.head(3))

# merge df_union into df_labor_all
df_labor_all = pd.merge(df_labor_all, df_union, on=['Month', 'State'], how='left')

# calculate union rate
df_labor_all['Union_Rate'] = df_labor_all['Union_Pop'] / df_labor_all['LaborForce_Pop'] * 100

df_labor_all.head(6)

Total Number of Union Members is:  168042639.0093
Shape of df_union is:  (612, 3)
All months and states are included in df_union
   Month  State    Union_Pop
0      1      1  150670.1065
1      1      2   53307.2985
2      1      4  120785.4938


,Month,State,Civil_Pop,LaborForce_Pop,Unemp_Pop,Emp_Pop,Emp_Rate,Unemp_Rate,LF_Particip_Rate,Emp_Pop_Ratio,Union_Pop,Union_Rate
0,1,1,3.890500e+06,2.216382e+06,9.859734e+04,2.117785e+06,95.551428,4.448572,56.969080,54.434770,1.506701e+05,6.798020
1,1,2,5.448940e+05,3.478068e+05,2.677084e+04,3.210359e+05,92.302955,7.697045,63.830171,58.917134,5.330730e+04,15.326699
2,1,4,5.932506e+06,3.554388e+06,2.375644e+05,3.316824e+06,93.316305,6.683695,59.913773,55.909319,1.207855e+05,3.398208
3,1,5,2.365706e+06,1.333109e+06,5.702834e+04,1.276081e+06,95.722156,4.277844,56.351442,53.940815,6.933877e+04,5.201281
4,1,6,3.109385e+07,1.861967e+07,1.715757e+06,1.690391e+07,90.785241,9.214759,59.882154,54.364158,2.303918e+06,12.373574
5,1,8,4.642794e+06,3.167476e+06,1.957061e+05,2.971770e+06,93.821386,6.178614,68.223485,64.008220,2.443588e+05,7.714623


The montly union info is not available, so we need to calculate the average annual union rate and verify.

Comparing with [BLS Union Tables](https://www.bls.gov/webapps/legacy/cpslutab5.htm), my calculation of Union_Pop is roughly (very roughly) the same, and the Union_Rate is much larger.

In [22]:
df_union_annual = df_labor_all.groupby(['Month']).sum(numeric_only=True)[['Union_Pop', 'Union_Rate']].reset_index()
df_union_annual['Union_Pop'] = df_union_annual['Union_Pop'] / 12
df_union_annual['Union_Rate'] = df_union_annual['Union_Rate'] / 12
df_union_annual.head(3)

,Month,Union_Pop,Union_Rate
0,1,1.178186e+06,35.260392
1,2,1.218056e+06,38.434145
2,3,1.180429e+06,34.818574


### 2.2 Demographics

If we want to know labor force rates in different demographic groups, we need to perform similar calculations as in 1.1 to 1.4, but with additional conditions.

I am not sure about what weight should be used if leave labor information out (only calculating percentage of female, white/black/asian, age group, etc). The weight to be used is second stage weight `PWSSWGT`, as described in data instruction page 6 "It is the most demographically correct weight" and data methodology page 74.

In [23]:
# calculate the total population (already done in CNP? do I need to keep weight consistent?)

## Step 3: Export Aggregated Data

In [24]:
df_labor_all.to_csv(GLOBAL_PATH + PROCESSED_DATA_PATH + str(CPS_YEAR) + '_aggregate.csv', index=False)